<a href="https://colab.research.google.com/github/mattocanas/ESM-Sequence-Generator/blob/main/ESM_Sequence_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing Dependencies

* Using biopython == 1.78 because newer versions don't seem to like Blosum Substitution Matrices

In [ ]:
%%capture
!pip install transformers datasets
!pip install biopython==1.78
!pip install colorama

## Tokenizer and Model

Using Meta's ESM2 Model. `t12_35M_UR50D` is the second smallest ESM2 model, with 12 layers, and 35 million parameters. Larger models will be more accurate, but also take up more RAM. However, since we are not fine-tuning the model yet, it would probably be worth it to try some of the other models out!




* esm2_t48_15B_UR50D
* esm2_t36_3B_UR50D
* esm2_t33_650M_UR50D
* esm2_t30_150M_UR50D
* esm2_t12_35M_UR50D
* esm2_t6_8M_UR50D

For more info on ESM2: https://www.biorxiv.org/content/10.1101/2022.07.20.500902v3.full

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t33_650M_UR50D")

model = AutoModelForMaskedLM.from_pretrained("facebook/esm2_t33_650M_UR50D")

## Library Generation Function

To get the same results each time, keep all the random seeds the same!

Want new results? Change the random seed, or comment it out.

The function generates 10 new versions of any stretch of amino acids that you specify. A few things to keep in mind:
* you can easily change the number of new functions generated by changing the integer here `for _ in range(10):`
* Python is a 0-indexed language. So if you have a starting and final position for the residues you want to generate a library for make sure you take into account the fact that the first residue is in position `0` and the second residue is position `1`, so on and so forth.
* To generate more random sequences, you can do a couple things:
   1. Increase the number of top predictions that the model chooses from for position 3-end. `top_k_logits = torch.topk(scaled_predictions[0, mask_position[0]], 2)`. Change the 2 to a 3 or 4.
   2. Increase the temperature above `1`. A higher temperature makes the difference between the top predictions smaller, so the algorithm is more likely to choose the 2nd or 3rd prediction. Inversely, if you decrease the temperature below `1`, the model will be more confident in its top prediction, so you will get less randomness. You can change the temperature by editing the following in the code block below: `temperature = 1`


**How it works:**

* For the span of amino acids you want to be generated, the first position is masked out, and the residue in that position is predicted by the model. The top prediction is taken from the model, and placed in the first position.
* The mask is then moved into the second position, and the top prediction for that mask is then taken from the model
* The mask is moved into the third position, and the model then randomly selects based on a distribution, between the top 2 predictions from the model for that position. The mask is then moved down and this continues until the end of the span.


In [ ]:
import torch
from torch.nn import functional as F
import random
import numpy as np

# Set a seed for random number generation
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

def generate_library(model, tokenizer, sequence, start_position, end_position):
    libraries = []
    for _ in range(20):
        sequence_copy = list(sequence)  # Use a fresh copy for each sequence
        mutated_sequence = []
        for pos in range(start_position, end_position + 1):
            sequence_copy[pos] = "<mask>"
            for i in range(pos, end_position + 1):
                sequence_copy[i] = "<mask>"
                input_sequence = "".join(sequence_copy)
                input = tokenizer.encode(input_sequence, return_tensors="pt")

                outputs = model(input)
                predictions = outputs.logits
                temperature = 2  # Adjust the temperature here
                scaled_predictions = predictions / temperature

                mask_position = torch.where(input == tokenizer.mask_token_id)[1]

                # Make sure the mask position is within bounds
                if len(mask_position) > 0 and mask_position[0] < scaled_predictions.shape[1]:
                    if i == start_position or i == start_position + 1:
                        # For the first two positions, use the top prediction
                        top_k_logits = torch.topk(scaled_predictions[0, mask_position[0]], 1)
                    else:
                        # For other positions, use the top 3 predictions
                        top_k_logits = torch.topk(scaled_predictions[0, mask_position[0]], 3)

                    top_probs = F.softmax(top_k_logits.values, dim=-1).detach().numpy()

                    # Select the prediction
                    selected_pred = np.random.choice(top_k_logits.indices.tolist(), p=top_probs.flatten())

                    # Decode the prediction and remove spaces
                    decoded_pred = tokenizer.decode([selected_pred]).replace(" ", "")

                    # Replace the masked position with the predicted token
                    sequence_copy[i] = decoded_pred

                if pos == i:  # Only append to the mutated sequence once for each position
                    mutated_sequence.append(decoded_pred)

        # Join the sequence in the specified range
        full_sequence = "".join(sequence_copy)
        mutated_sequence = "".join(mutated_sequence)
        libraries.append((full_sequence, mutated_sequence))

    return libraries




## Sequence and Positions

Input your protein sequence here!

Also, specify the starting and final amino acid position for the span you want to be generated! Remember that the string is 0-indexed, so start counting from 0!

In [ ]:
sequence = "QVQLVQSGAEVRKPGASVKVSCKASGYSFTDYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS"
start_position = 25
end_position = 33

libraries = generate_library(model, tokenizer, sequence, start_position, end_position)
for full_sequence, mutated_sequence in libraries:
    print(f"Full sequence: {full_sequence}\nCDR/Library portion: {mutated_sequence}\n")

Full sequence: QVQLVQSGAEVRKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
CDR/Library portion: GYTFTGYYM

Full sequence: QVQLVQSGAEVRKPGASVKVSCKASGYSMSGYWIHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
CDR/Library portion: GYSMSGYWI

Full sequence: QVQLVQSGAEVRKPGASVKVSCKASGYTFSSYYVHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
CDR/Library portion: GYTFSSYYV

Full sequence: QVQLVQSGAEVRKPGASVKVSCKASGYTFSDYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
CDR/Library portion: GYTFSDYYM

Full sequence: QVQLVQSGAEVRKPGASVKVSCKASGYSFSSHYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
CDR/Library portion: GYSFSSHYM

Full sequence: QVQLVQSGAEVRKPGASVKVSCKASGYSLSDAWMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
CDR/Library portion: GYSLSDAWM

## Comparison to IgLm

IgLm is a protein large language model from the Gray lab: https://github.com/Graylab/IgLM

IgLm is a fine-tuned GPT-2 model on the OAS. It can generate full sequences and infill spans. While this current method is not capable of producing full length sequences (yet), it can do infilling of shorter spans given enough context. Thus, I wanted to get a rough comparison between this model and IgLm for the same infilling. The list of sequences below were the ones generated by IgLm for the same span that I was using earlier in this notebook.

Below is a simple MSA for the sequences generated in this model, and those generated by IgLm. You can definitely see overlap!

Further below are two scores using a Blosum62 matrix. Higher scores are better! The first cell is the average score between the library generated in this model, and those generated from IgLm. The last cell is the average score between our infills randomly shuffled, and the IgLm sequences. You can see that this models score is significantly higher than random generation. So there is definitely overlap!

In [ ]:
from colorama import Fore, Style
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord


# Let's say `full_sequences` is your list of all full sequences
full_sequences = [seq[0] for seq in libraries]

#Sequences generated from IgLm
iglm_sequences = ['QVQLVQSGAEVRKPGASVKVSCKASGYTFSDYDMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTGHYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYSFSGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFIDHYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTAKHMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTDHHMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFSGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTGSYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYIFIGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYMFTGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYMFSDYFMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYNFIDYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYIFTGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYIFTDYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTDYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTEYFMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTLTGSYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGDTFSGFYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  ]

# Convert each string in the libraries list to a SeqRecord
seqrecords_library = [SeqRecord(Seq(seq)) for seq in full_sequences]

# Convert each string in the IgLm list to a SeqRecord
seqrecords_iglm = [SeqRecord(Seq(seq)) for seq in iglm_sequences]

# Create a multiple sequence alignment object for each list
msa_library = MultipleSeqAlignment(seqrecords_library)
msa_iglm = MultipleSeqAlignment(seqrecords_iglm)

# Function to print sequences with color formatting
def print_sequences(msa):
    # Iterate over each SeqRecord in the alignment
    for record in msa:
        # Split the sequence into colored and uncolored segments
        uncolored_start = record.seq[:start_position]
        colored = record.seq[start_position:end_position]
        uncolored_end = record.seq[end_position:]

        # Construct the output string with color codes
        output = str(uncolored_start) + Fore.RED + str(colored) + Style.RESET_ALL + str(uncolored_end)
        print(output)

# Print library sequences
print("Library sequences:")
print_sequences(msa_library)

# Print dashed line
print('-'*50)

# Print IgLm sequences
print("IgIm sequences:")
print_sequences(msa_iglm)


Library sequences:
QVQLVQSGAEVRKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
QVQLVQSGAEVRKPGASVKVSCKASGYSMSGYWIHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
QVQLVQSGAEVRKPGASVKVSCKASGYTFSSYYVHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
QVQLVQSGAEVRKPGASVKVSCKASGYTFSDYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
QVQLVQSGAEVRKPGASVKVSCKASGYSFSSHYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
QVQLVQSGAEVRKPGASVKVSCKASGYSLSDAWMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
QVQLVQSGAEVRKPGASVKVSCKASGYTFTNYYIHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
QVQLVQSGAEVRKPGASVKVSCKASGYTFTDYWMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS
QVQLVQSGAEVRKPGASVKVS

In [ ]:
import random
from Bio import pairwise2
from Bio.SubsMat import MatrixInfo as matlist

# get the BLOSUM62 matrix
matrix = matlist.blosum62

# let's assume you have your sequences in two lists
sequences = [seq[0] for seq in libraries]
iglm_seqs = ['QVQLVQSGAEVRKPGASVKVSCKASGYTFSDYDMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTGHYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYSFSGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFIDHYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTAKHMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTDHHMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFSGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTGSYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYIFIGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYMFTGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYMFSDYFMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYNFIDYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYIFTGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYIFTDYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTDYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTEYFMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTLTGSYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGDTFSGFYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  ]

# initialize the total score and the number of alignments
total_score = 0
num_alignments = 0

# iterate over all pairs of sequences in the two lists
for seq1 in sequences:
    for seq2 in iglm_seqs:
        # Extract the desired positions (25-33) from each sequence
        seq1_subset = seq1[start_position - 1:end_position]
        seq2_subset = seq2[start_position - 1:end_position]

        # align the sequences using the BLOSUM62 matrix
        # gap opening penalty is -10, gap extension penalty is -0.5
        alignment = pairwise2.align.globalds(seq1_subset, seq2_subset, matrix, -10, -0.5, score_only=True)

        # add the score to the total and increment the number of alignments
        total_score += alignment
        num_alignments += 1

# calculate the average score
average_score = total_score / num_alignments

print(f"Average alignment score: {average_score}")


Average alignment score: 32.1825


/usr/local/lib/python3.10/dist-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [ ]:
import random
from Bio import pairwise2
from Bio.SubsMat import MatrixInfo as matlist

# get the BLOSUM62 matrix
matrix = matlist.blosum62

# let's assume you have your sequences in two lists
sequences = [seq[0] for seq in libraries]

# shuffle positions 25-33 in each sequence
sequences = [seq[:start_position -1] + ''.join(random.sample(seq[start_position -1:end_position], len(seq[start_position -1:end_position]))) + seq[end_position:] for seq in sequences]

# define the standard sequences
iglm_seqs = ['QVQLVQSGAEVRKPGASVKVSCKASGYTFSDYDMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTGHYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYSFSGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFIDHYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTAKHMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTDHHMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFSGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTGSYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYIFIGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYMFTGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYMFSDYFMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYNFIDYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYIFTGYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYIFTDYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTDYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTEYFMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTLTGSYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGYTFTSYYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  'QVQLVQSGAEVRKPGASVKVSCKASGDTFSGFYMHWVRQAPGQGLEWMGWINPKSGGTNYAQRFQGRVTMTGDTSISAAYMDLASLTSDDTAVYYCVKDCGSGGLRDFWGQGTTVTVSS',
                  ]


# initialize the total score and the number of alignments
total_score = 0
num_alignments = 0

# iterate over all pairs of sequences in the two lists
for seq1 in sequences:
    for seq2 in iglm_seqs:
        # align only the specified region of the sequences using the BLOSUM62 matrix
        # gap opening penalty is -10, gap extension penalty is -0.5
        alignment = pairwise2.align.globalds(seq1[start_position - 1:end_position], seq2[start_position -1:end_position], matrix, -10, -0.5, score_only=True)

        # add the score to the total and increment the number of alignments
        total_score += alignment
        num_alignments += 1

# calculate the average score
average_score = total_score / num_alignments

print(f"Average alignment score: {average_score}")


Average alignment score: -0.4475
